In [2]:
import face_recognition as fr
import cv2
import glob
import numpy as np
import os
import mediapipe as mp
import random
from sklearn.multiclass import OneVsRestClassifier

In [11]:
"""
...
...
Run this cell to load training data
...
...
"""
data = "../lfw/*/"
x = []
y = []
classes = {}
#num_classes = len(glob.glob(os.path.join(data)))
#n = random.randint(0, num_classes)
for idx, folder in enumerate(glob.glob(os.path.join(data))):
    if idx == 2000:
        break
    num_pics = len(glob.glob(os.path.join(folder+"*")))
    if num_pics < 10:
        continue
    use = glob.glob(os.path.join(folder+"*"))[:10]
    person_name = folder.split("\\")[1]
    print(idx, person_name)
    classes[idx] = person_name
    for f in use:
        pic = cv2.imread(f)
        detected_pos = fr.face_locations(pic)
        for i in detected_pos:
            top, right, bottom, left = i
            face_image = pic[top:bottom, left:right]
            try:
                face_image = cv2.resize(face_image, (256, 256))
                face_embedding = fr.face_encodings(face_image)[0]
                x.append(face_embedding)
                y.append(idx)
            except:
                continue
print(len(x))

17 Abdullah_Gul
50 Adrien_Brody
121 Alejandro_Toledo
204 Alvaro_Uribe
223 Amelie_Mauresmo
272 Andre_Agassi
290 Andy_Roddick
302 Angelina_Jolie
317 Anna_Kournikova
335 Ann_Veneman
370 Ariel_Sharon
374 Ari_Fleischer
386 Arnold_Schwarzenegger
417 Atal_Bihari_Vajpayee
544 Bill_Clinton
551 Bill_Gates
562 Bill_McBride
573 Bill_Simon
706 Britney_Spears
781 Carlos_Menem
783 Carlos_Moya
831 Catherine_Zeta-Jones
875 Charles_Moose
1047 Colin_Powell
1059 Condoleezza_Rice
1207 David_Beckham
1252 David_Nalbandian
1347 Dick_Cheney
1382 Dominique_de_Villepin
1393 Donald_Rumsfeld
1473 Edmund_Stoiber
1476 Eduardo_Duhalde
1704 Fidel_Castro
1854 George_HW_Bush
1865 George_Robertson
1870 George_W_Bush
1891 Gerhard_Schroeder
1932 Gloria_Macapagal_Arroyo
1940 Gonzalo_Sanchez_de_Lozada
1945 Gordon_Brown
1964 Gray_Davis
1993 Guillermo_Coria
426


In [93]:
import json 
with open("classes.json", "w") as outfile:
    json.dump(classes, outfile)

In [3]:
import json
class_f = open('classes.json')
classes = json.load(class_f)
print(dict(classes))

{'17': 'Abdullah_Gul', '50': 'Adrien_Brody', '121': 'Alejandro_Toledo', '204': 'Alvaro_Uribe', '223': 'Amelie_Mauresmo', '272': 'Andre_Agassi', '290': 'Andy_Roddick', '302': 'Angelina_Jolie', '317': 'Anna_Kournikova', '335': 'Ann_Veneman', '370': 'Ariel_Sharon', '374': 'Ari_Fleischer', '386': 'Arnold_Schwarzenegger', '417': 'Atal_Bihari_Vajpayee', '544': 'Bill_Clinton', '551': 'Bill_Gates', '562': 'Bill_McBride', '573': 'Bill_Simon', '706': 'Britney_Spears', '781': 'Carlos_Menem', '783': 'Carlos_Moya', '831': 'Catherine_Zeta-Jones', '875': 'Charles_Moose', '1047': 'Colin_Powell', '1059': 'Condoleezza_Rice', '1207': 'David_Beckham', '1252': 'David_Nalbandian', '1347': 'Dick_Cheney', '1382': 'Dominique_de_Villepin', '1393': 'Donald_Rumsfeld', '1473': 'Edmund_Stoiber', '1476': 'Eduardo_Duhalde', '1704': 'Fidel_Castro', '1854': 'George_HW_Bush', '1865': 'George_Robertson', '1870': 'George_W_Bush', '1891': 'Gerhard_Schroeder', '1932': 'Gloria_Macapagal_Arroyo', '1940': 'Gonzalo_Sanchez_de_L

In [4]:
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer
from sklearn.svm import SVC
from joblib import dump, load
import pickle

#model = OneVsRestClassifier(SVC(kernel='linear', probability=True, verbose=True, max_iter=1000))
#model.fit(x, y)
#pickle.dump(model, open("./model.pickle", 'wb'))

In [5]:
model = pickle.load(open("./model.pickle",'rb'))

In [6]:
model.classes_

array([  17,   50,  121,  204,  223,  272,  290,  302,  317,  335,  370,
        374,  386,  417,  544,  551,  562,  573,  706,  781,  783,  831,
        875, 1047, 1059, 1207, 1252, 1347, 1382, 1393, 1473, 1476, 1704,
       1854, 1865, 1870, 1891, 1932, 1940, 1945, 1964, 1993])

In [7]:
class_list = list(classes.keys())
print(len(class_list))
print(class_list)

42
['17', '50', '121', '204', '223', '272', '290', '302', '317', '335', '370', '374', '386', '417', '544', '551', '562', '573', '706', '781', '783', '831', '875', '1047', '1059', '1207', '1252', '1347', '1382', '1393', '1473', '1476', '1704', '1854', '1865', '1870', '1891', '1932', '1940', '1945', '1964', '1993']


### Here we go one by one through each test face image

In [42]:
data = "./archive/new_val/*/"
#data = "../lfw/*/"
for idx, folder in enumerate(glob.glob(os.path.join(data))):
    use = glob.glob(os.path.join(folder+"*"))
    person_name = folder.split("\\")[1]
    print(idx, person_name)
    #classes[idx] = person_name
    for f in use:
        pic = cv2.imread(f)
        try:
            face_embedding = fr.face_encodings(pic)[0]
            test = np.asarray(face_embedding)
            test=  np.reshape(test, (1, -1))
            pred = model.predict_proba(test)[0]
            if np.max(pred) > 0.7:
                print(classes[str(class_list[np.argmax(pred)])], np.max(pred))
            elif np.max(pred) > 0.5 :
                #added to see what it predicts when highest probability is quite low
                #print(classes[str(class_list[np.argmax(pred)])], np.max(pred)) 
                choose_folder = "../lfw\\"+classes[str(class_list[np.argmax(pred)])]+"\\*"
                test_pic_path = glob.glob(os.path.join(choose_folder))[0]
                test_pic = cv2.imread(test_pic_path)
                try:
                    test_face_embedding = fr.face_encodings(test_pic)[0]
                    results = fr.compare_faces([face_embedding], test_face_embedding)
                    if results[0]:
                        print(classes[str(class_list[np.argmax(pred)])], np.max(pred)) 
                    else:
                        print("Face not found!")
                except Exception as first_e:
                    raise(first_e)
            else:
                print("Face not found!")
                    
        except Exception as e:
            raise(e)
            continue
            

0 Angelina_Jolie
Angelina_Jolie 0.847412496311433
Angelina_Jolie 0.8555918807050934
1 Arnold_Schwarzenegger
Arnold_Schwarzenegger 0.8567437045431145
2 Atal_Bihari_Vajpayee
Atal_Bihari_Vajpayee 0.7583958862424843
Atal_Bihari_Vajpayee 0.8698240790668677
3 Ben_Affleck
Face not found!
Face not Found!
Face not Found!
Face not Found!
Face not found!
Face not Found!
Face not found!
4 Bill_Clinton
Bill_Clinton 0.9019500973477823
Bill_Clinton 0.7402862380662191
Bill_Clinton 0.7530243015160745
5 Bill_Gates
Bill_Gates 0.8517347508201452
Bill_Gates 0.8240598993401298
6 Britney_Spears
Britney_Spears 0.8117965004096812
Britney_Spears 0.8157516969089398
Britney_Spears 0.795923989123969
7 Charles_Moose
Charles_Moose 0.8244098826157716
Charles_Moose 0.8985889341828631
Charles_Moose 0.8874299673427328
8 David_Beckham
David_Beckham
9 Fidel_Castro
Fidel_Castro 0.7560513208829484
10 George_HW_Bush
George_HW_Bush 0.8190898254840092
11 George_W_Bush
George_W_Bush 0.8252153657441565
George_W_Bush 0.8993389551

### Here we input all face embeddings in a single array - gives faster results

In [8]:
data = "./archive/new_val/*/"
testx = []
for idx, folder in enumerate(glob.glob(os.path.join(data))):
    use = glob.glob(os.path.join(folder+"*"))
    person_name = folder.split("\\")[1]
    print(idx, person_name)
    for f in use:
        pic = cv2.imread(f)
        try:
            face_embedding = fr.face_encodings(pic)[0]
            test = np.asarray(face_embedding)
            
            testx.append(test)
        except:
            continue


0 Angelina_Jolie
1 Arnold_Schwarzenegger
2 Atal_Bihari_Vajpayee
3 Ben_Affleck
4 Bill_Clinton
5 Bill_Gates
6 Britney_Spears
7 Charles_Moose
8 David_Beckham
9 Fidel_Castro
10 George_HW_Bush
11 George_W_Bush


In [10]:
probs = model.predict_proba(testx)
for idx, pred in enumerate(probs):
    if np.max(pred) > 0.7:
        print(classes[str(class_list[np.argmax(pred)])], np.max(pred))
    elif np.max(pred) > 0.5:
        choose_folder = "../lfw\\"+classes[str(class_list[np.argmax(pred)])]+"\\*"
        real_face_embedding = testx[idx]
        test_pic_path = glob.glob(os.path.join(choose_folder))[0]
        test_pic = cv2.imread(test_pic_path)
        try:
            test_face_embedding = fr.face_encodings(test_pic)[0]
            results = fr.compare_faces([real_face_embedding], test_face_embedding)
            if results[0]:
                print(classes[str(class_list[np.argmax(pred)])], np.max(pred)) 
            else:
                print("Face not found!")
        except Exception as first_e:
            print("Face not found!")
    else:
        print("Face not found!")

Angelina_Jolie 0.8474124963114331
Angelina_Jolie 0.8555918807050937
Arnold_Schwarzenegger 0.8567437045431144
Atal_Bihari_Vajpayee 0.7583958862424842
Atal_Bihari_Vajpayee 0.8698240790668677
Face not found!
Face not found!
Face not found!
Face not found!
Face not found!
Face not found!
Face not found!
Bill_Clinton 0.9019500973477825
Bill_Clinton 0.740286238066219
Bill_Clinton 0.7530243015160747
Bill_Gates 0.8517347508201448
Bill_Gates 0.8240598993401296
Britney_Spears 0.8117965004096811
Britney_Spears 0.8157516969089399
Britney_Spears 0.7959239891239691
Charles_Moose 0.8244098826157716
Charles_Moose 0.898588934182863
Charles_Moose 0.8874299673427327
David_Beckham 0.5144691863901363
Fidel_Castro 0.7560513208829485
George_HW_Bush 0.8190898254840092
George_W_Bush 0.8252153657441567
George_W_Bush 0.8993389551450134
